# NumPy数组的计算：通用函数

`NumPy` 在 `Python` 数据科学世界中如此重要的原因明确点说，`NumPy` 提供了一个简单灵活的接口来优化数据数组的计算。  
`NumPy` 数组的计算有时非常快，有时也非常慢。使 `NumPy` 变快的关键是利用向量化操作，通常在 `NumPy` 的通用函数（`ufunc`）中实现。  
本节将介绍 `NumPy` 通用函数的重要性——它可以提高数组元素的重复计算的效率；然后，将会介绍很多 `NumPy` 包中常用且有用的数学通用函数。

In [1]:
import numpy as np

np.random.seed(0)

In [2]:
%%html
<style>
  table {margin-left: 0 !important;}
</style>

## 1. 缓慢的循环

`Python` 的默认实现（被称作 `CPython`）处理起某些操作时非常慢，一部分原因是该语言的动态性和解释性——数据类型灵活的特性决定了序列操作不能像 `C` 语言和 `Fortran` 语言一样被编译成有效的机器码。  
目前，有一些项目试图解决 `Python` 这一弱点，比较知名的包括：
- `PyPy` 项目[http://pypy.org/](http://pypy.org/)，一个实时的 `Python` 编译实现；
- `Cython` 项目[http://cython.org](http://cython.org)，将 `Python` 代码转换成可编译的 `C` 代码；
- `Numba`项目[http://numba.pydata.org/](http://numba.pydata.org/)，将 `Python` 代码的片段转换成快速的 `LLVM` 字节码。

以上这些项目都各有其优势和劣势，但是比较保守地说，这些方法中还没有一种能达到或超过标准 `CPython` 引擎的受欢迎程度。

In [3]:
# Python 的相对缓慢通常出现在很多小操作需要不断重复的时候，比如对数组的每个元素做循环操作时。
# 假设有一个数组，我们想计算每个元素的倒数，一种直接的解决方法如下，但是如果测试一个很大量的输入数据运行上述代码的时间，这一操作将非常耗时，并且是超出意料的慢！
def compute_reciprocals(values):
    output = np.empty(len(values))
    for i in range(len(values)):
        output[i] = 1.0 / values[i]
    return output

big_array = np.random.randint(1, 100, size=1000000)
%timeit compute_reciprocals(big_array)
# 完成百万次上述操作并存储结果花了几秒钟的时间！在手机都以 Giga-FLOPS（即每秒十亿次浮点运算）为单位计算处理速度时，上面的处理结果所花费的时间确实是不合时宜的慢。
# 事实上，这里的处理瓶颈并不是运算本身，而是 CPython 在每次循环时必须做数据类型的检查和函数的调度。每次进行倒数运算时，Python 首先检查对象的类型，并且动态查找可以使用该数据类型的正确函数。
# 如果我们在编译代码时进行这样的操作，那么就能在代码执行之前知晓类型的声明，结果的计算也会更加有效率。

5.83 s ± 90.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## 2. 通用函数介绍

`NumPy` 为很多类型的操作提供了非常方便的、静态类型的、可编译程序的接口，也被称作向量操作。  
你可以通过简单地对数组执行操作来实现，这里对数组的操作将会被用于数组中的每一个元素。  
这种向量方法被用于将循环推送至 `NumPy` 之下的编译层，这样会取得更快的执行效率。

`NumPy` 中的向量操作是通过通用函数实现的。通用函数的主要目的是对 `NumPy` 数组中的值执行更快的重复操作。

In [4]:
# 比较下面两个结果
values = np.random.randint(1, 10, size=5)
print(compute_reciprocals(values))
print(1.0/values)

[0.25       0.16666667 0.14285714 0.33333333 0.16666667]
[0.25       0.16666667 0.14285714 0.33333333 0.16666667]


In [5]:
%timeit (1.0/big_array)  # 如果计算一个较大数组的运行时间，可以看到它的完成时间比 Python 循环花费的时间更短

7.35 ms ± 225 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
np.arange(5) / np.arange(1, 6)  # 向量操作还可以对两个数组进行计算

array([0.        , 0.5       , 0.66666667, 0.75      , 0.8       ])

In [7]:
x = np.arange(9).reshape((3,3))  # 向量操作还可以对多维数组进行计算
2 ** x

array([[  1,   2,   4],
       [  8,  16,  32],
       [ 64, 128, 256]])

通过通用函数用向量的方式进行计算几乎总比用 `Python` 循环实现的计算更加有效，尤其是当数组很大时。  
只要你看到 `Python` 脚本中有这样的循环，就应该考虑能否用向量方式替换这个循环。

## 3. 探索NumPy的通用函数

通用函数有两种存在形式：
- **一元通用函数**（unary ufunc）对单个输入操作
- **二元通用函数**（binary ufunc）对两个输入操作

### 3.1. 数组的运算

In [8]:
# NumPy 通用函数的使用方式非常自然，因为它用到了 Python 原生的算术运算符，标准的加、减、乘、除、逻辑非、** 表示的指数运算符和 % 表示的模运算符都可以使用
x = np.arange(4)
print("x     =", x)
print("x + 5 =", x + 5)
print("x - 5 =", x - 5)
print("x * 2 =", x * 2)
print("x / 2 =", x / 2)
print("x // 2 =", x // 2)
print("-x     = ", -x)
print("x ** 2 = ", x ** 2)
print("x % 2  = ", x % 2)

x     = [0 1 2 3]
x + 5 = [5 6 7 8]
x - 5 = [-5 -4 -3 -2]
x * 2 = [0 2 4 6]
x / 2 = [0.  0.5 1.  1.5]
x // 2 = [0 0 1 1]
-x     =  [ 0 -1 -2 -3]
x ** 2 =  [0 1 4 9]
x % 2  =  [0 1 0 1]


In [9]:
-(0.5*x + 1) ** 2

array([-1.  , -2.25, -4.  , -6.25])

In [10]:
np.add(x, 2)  # 所有这些算术运算符都是 NumPy 内置函数的简单封装器，例如 + 运算符就是一个 add 函数的封装器

array([2, 3, 4, 5])

**`NumPy`实现的算术运算符**

| 运算符 | 对应的通用函数 | 描述 |
| -- | -- | -- |
| + | np.add | 加法运算（即 1 + 1 = 2） |
| - | np.subtract | 减法运算（即 3 - 2 = 1） |
| - | np.negative | 负数运算（ 即 -2） |
| * | np.multiply | 乘法运算（即 2 \* 3 = 6） |
| / | np.divide | 除法运算（即 3 / 2 = 1.5） |
| // | np.floor_divide | 地板除法运算（floor division，即 3 // 2 = 1） |
| ** | np.power | 指数运算（即 2 ** 3 = 8） |
| % | np.mod | 模 / 余数（ 即 9 % 4 = 1） |

另外，`NumPy` 中还有布尔 / 位运算符，这些运算符将在 2.6 节中进一步介绍。

### 3.2. 绝对值

In [11]:
x = np.array([-2, -1, 0, 1, 2])  # 正如 NumPy 能理解 Python 内置的运算操作，NumPy 也可以理解Python 内置的绝对值函数
abs(x)

array([2, 1, 0, 1, 2])

In [12]:
print(np.absolute(x))  # 对应的 NumPy 通用函数是 np.absolute，该函数也可以用别名np.abs 来访问
print(np.abs(x))

[2 1 0 1 2]
[2 1 0 1 2]


In [13]:
x = np.array([3-4j, 4-3j, 2+0j, 0+1j])  # 这个通用函数也可以处理复数。当处理复数时，绝对值返回的是该复数的模（magnitude）
np.abs(x)

array([5., 5., 2., 1.])

### 3.3. 三角函数

In [14]:
# NumPy 提供了大量好用的通用函数，其中对于数据科学家最有用的就是三角函数。首先定义一个角度数组：
theta = np.linspace(0, np.pi, 3)

print("theta      = ", theta)
print("sin(theta) = ", np.sin(theta))
print("cos(theta) = ", np.cos(theta))
print("tan(theta) = ", np.tan(theta))
# 这些值是在机器精度内计算的，所以有些应该是 0 的值并没有精确到0

theta      =  [0.         1.57079633 3.14159265]
sin(theta) =  [0.0000000e+00 1.0000000e+00 1.2246468e-16]
cos(theta) =  [ 1.000000e+00  6.123234e-17 -1.000000e+00]
tan(theta) =  [ 0.00000000e+00  1.63312394e+16 -1.22464680e-16]


In [15]:
# 逆三角函数
x = [-1, 0, 1]
print("x         = ", x)
print("arcsin(x) = ", np.arcsin(x))
print("arccos(x) = ", np.arccos(x))
print("arctan(x) = ", np.arctan(x))

x         =  [-1, 0, 1]
arcsin(x) =  [-1.57079633  0.          1.57079633]
arccos(x) =  [3.14159265 1.57079633 0.        ]
arctan(x) =  [-0.78539816  0.          0.78539816]


### 3.4. 指数和对数

In [16]:
x = [1, 2, 3]
print("x     =", x)
print("e^x   =", np.exp(x))
print("2^x   =", np.exp2(x))
print("3^x   =", np.power(3, x))

x     = [1, 2, 3]
e^x   = [ 2.71828183  7.3890561  20.08553692]
2^x   = [2. 4. 8.]
3^x   = [ 3  9 27]


In [17]:
x = [1, 2, 4, 10]
print("x        =", x)
print("ln(x)    =", np.log(x))
print("log2(x)  =", np.log2(x))
print("log10(x) =", np.log10(x))

x        = [1, 2, 4, 10]
ln(x)    = [0.         0.69314718 1.38629436 2.30258509]
log2(x)  = [0.         1.         2.         3.32192809]
log10(x) = [0.         0.30103    0.60205999 1.        ]


In [18]:
# 还有一些特殊的版本，对于非常小的输入值可以保持较好的精度：
# 当 x 的值很小时，以下函数给出的值比 np.log 和 np.exp 的计算更精确。
x = [0, 0.001, 0.01, 0.1]
print("exp(x) - 1 =", np.expm1(x))
print("log(1 + x) =", np.log1p(x))

exp(x) - 1 = [0.         0.0010005  0.01005017 0.10517092]
log(1 + x) = [0.         0.0009995  0.00995033 0.09531018]


### 3.5. 专用的通用函数

除了以上介绍到的，`NumPy` 还提供了很多通用函数，包括双曲三角函数、比特位运算、比较运算符、弧度转化为角度的运算、取整和求余运算，等等。浏览 `NumPy` 的文档将会揭示很多有趣的功能。  
还有一个更加专用，也更加晦涩的通用函数优异来源是子模块`scipy.special`。如果你希望对你的数据进行一些更晦涩的数学计算，`scipy.special` 可能包含了你需要的计算函数，这些函数能列一个长长的列表。  
下面的代码片段展示了一些可能在统计学中用到的函数：

In [19]:
from scipy import special

In [20]:
# Gamma函数（广义阶乘，generalized factorials）和相关函数
x = [1, 5, 10]
print("gamma(x)     =", special.gamma(x))
print("ln|gamma(x)| =", special.gammaln(x))
print("beta(x, 2)   =", special.beta(x, 2))

gamma(x)     = [1.0000e+00 2.4000e+01 3.6288e+05]
ln|gamma(x)| = [ 0.          3.17805383 12.80182748]
beta(x, 2)   = [0.5        0.03333333 0.00909091]


In [21]:
# 误差函数（高斯积分）
# 它的实现和它的逆实现
x = np.array([0, 0.3, 0.7, 1.0])
print("erf(x)  =", special.erf(x))
print("erfc(x) =", special.erfc(x))
print("erfinv(x) =", special.erfinv(x))

erf(x)  = [0.         0.32862676 0.67780119 0.84270079]
erfc(x) = [1.         0.67137324 0.32219881 0.15729921]
erfinv(x) = [0.         0.27246271 0.73286908        inf]


## 4. 高级的通用函数特性

### 4.1. 指定输出

在进行大量运算时，有时候指定一个用于存放运算结果的数组是非常有用的。不同于创建临时数组，你可以用这个特性将计算结果直接写入到你期望的存储位置。  
所有的通用函数都可以通过 `out` 参数来指定计算结果的存放位置：